In [1]:
import pandas as pd
import alphalens as al
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
from functools import partial

# <center>winsorize函数没改进，adjust函数进行了改进</center>

## 1.winsorize

1. **先执行先验去极值的操作：** 根据输入向量x的分位数来判断，将分布在0.1%和99.9%之外的数值替换为空值。
2. **接下来的部分使用3倍标准差法去极值化：** 计算x的均值 mean 和标准差。通过判断每个值是否超出了均值加减3倍标准差的范围，并将超出范围的值替换为对应的上下限值。

In [2]:
def winsorize(x, method):   
    '''去极值化函数，输入原始向量，输出去极值化后的factor、pct、cmv和industry（向量）'''
    
    # 先验去极值
    if method == 'factor':
        alpha = 0.002
        up = x.quantile(1 - alpha / 2)
        down = x.quantile(alpha / 2)
        x = np.where((x > down) & (x < up), x, np.nan)  #先验信息，0.1%和99.9%之外的factor值改为np.nan
        #x = np.where(x > 0, x, np.nan)  #对于某些财务因子，如PE和PB，去掉负值
    else:
        pass
    
    # 3-sigma去极值  
    mean = np.mean(x)
    std = np.std(x)
    result = np.where(x > mean+3*std, mean+3*std, x)
    result = np.where(x < mean-3*std, mean-3*std, result)
    return result

## 2.process_day

1. **获取当天数据：** 从传入的data数据中，通过loc方法获取指定的数据。 
2. **去极值化操作：** 使用winsorize先验去极值处理。
3. **处理缺失值：** 删除包含缺失值的行。
4. **平滑处理零值：** 对factor零值进行平滑处理，添加1。并检查factor和tcap列的标准差是否为零，如果是则使用平方根函数对factor处理。
5. **Z-score标准化：** 计算factor和tcap列的均值和标准差，标准化处理。
6. **生成行业哑变量**
7. **回归中性化：** 创建一个回归模型，对回归模型进行拟合，获取残差值作为新的因子值。
8. **重置索引**

In [3]:
# 去极值化、标准化和回归
# def process_day(day, data):
#     da = data.loc[day].copy()
#     # 去掉0到100之外的数值和3-sigma去极值
#     da['factor'] = winsorize(da['factor'], 'factor')   
#     da.dropna(inplace=True)
#     # 标准化
#     da[['factor', 'tcap']] = ((da[['factor', 'tcap']] - da[['factor', 'tcap']].mean()) / da[['factor', 'tcap']].std()).values
#     # 行业哑变量
#     industry_day = pd.get_dummies(da['first_industry_code'])  
#     # 回归
#     X = np.column_stack((da['tcap'], industry_day))
#     X = sm.add_constant(X)
#     model = sm.OLS(da['factor'], X)
#     result = model.fit()
#     # 残差值作为新的因子值
#     da['factor'] = result.resid  
#     da = da[['asset', 'factor']].reset_index()
#     return da

# 去极值化、标准化和回归
def process_day(day, data):
    da = data.loc[day].copy()
    # 去掉0到100之外的数值和3-sigma去极值
    da['factor'] = winsorize(da['factor'], 'factor')   
    da.dropna(inplace=True)
    # 平滑处理零值+1
    da['factor'] = da['factor'] + 1
    
    # 标准化
    std_factor_tcap = da[['factor', 'tcap']].std()
    if std_factor_tcap.equals(0):
        # 平方根处理
        da['factor'] = np.sqrt(da['factor'])  

    da[['factor', 'tcap']] = ((da[['factor', 'tcap']] - da[['factor', 'tcap']].mean()) / da[['factor', 'tcap']].std()).values
    # 行业哑变量
    industry_day = pd.get_dummies(da['first_industry_code'])  
    # 回归
    X = np.column_stack((da['tcap'], industry_day))
    X = sm.add_constant(X)
    model = sm.OLS(da['factor'], X)
    result = model.fit()
    # 残差值作为新的因子值
    da['factor'] = result.resid  
    da = da[['asset', 'factor']].reset_index()
    return da

## 3.adjust2

1. **日期筛选和排序：** data按照日期设为索引然后排序。
2. **获取所有交易日**
3. **处理数据：** 使用map和partial对每一个交易日应用process_day函数，传入对应日期的数据和整体数据集data。每个交易日的数据处理结果被收集到一个列表中，并合并成一个调整因子。
4. **左连接返回结果**

In [4]:
def adjust2(data):
    # 筛选日期
    data.set_index('date', inplace=True)
    data.sort_index(inplace=True)
    # 获取所有交易日
    days = sorted(set(data.index))
    # 处理
    process_day_partial = partial(process_day, data=data)
    adjusted_factor = pd.concat(map(process_day_partial, days), ignore_index=True)

    data.reset_index(inplace=True)
    
    return adjusted_factor.merge(data[['asset', 'date']], how='left', on=['asset', 'date'])